# Creator & Information Expert

Dwie zasady odpowiadające na pytanie: **Kto powinien tworzyć obiekty?**

## Creator

**Obiekt A powinien tworzyć obiekt B jeżeli**:
- A składa się (komponuje) z B, lub
- A zapisuje B do pliku/bazy, lub
- A używa B, lub
- A posiada wszystkie dane potrzebne do stworzenia B

### Źle: Tworzenie na zewnątrz

In [2]:
class Wheel:
    def __init__(self, width):
        self.__width = width


class Frame:
    def __init__(self, length):
        self.__length = length


class Bike:
    def __init__(self, wheel, frame):
        self.__wheel = wheel
        self.__frame = frame


In [3]:
# Klient musi tworzyć wszystkie części
wheel = Wheel(width=14)
frame = Frame(length=50)
bike = Bike(wheel=wheel, frame=frame)

**Problem**: 
- Klient musi znać implementację `Wheel` i `Frame`
- Klient musi wiedzieć, jak zbudować rower
- 3 linie kodu zamiast 1

### Dobrze: Tworzenie wewnątrz

In [4]:
class Wheel:
    def __init__(self, width):
        self.__width = width

class Frame:
    def __init__(self, length):
        self.__length = length

class Bike:
    def __init__(self, wheel_width, frame_length):
        self.__wheel = Wheel(wheel_width)  # <- Bike tworzy Wheel
        self.__frame = Frame(frame_length)  # <- Bike tworzy Frame


In [5]:
# Klient dostarcza tylko parametry
bike = Bike(wheel_width=14, frame_length=50)

**Korzyści**:
- Klient nie musi znać `Wheel` i `Frame`
- Prostsze API (1 linia zamiast 3)
- Bike kontroluje proces tworzenia

**Kiedy zastosować Creator?**
- Bike **składa się** z Wheel i Frame - Bike je tworzy
- Bike **posiada dane** (wheel_width, frame_length) - Bike je tworzy

## Information Expert

**Obiekt A może tworzyć obiekt B tylko jeśli A jest "ekspertem" w tworzeniu B.**

"Ekspert" = posiada wszystkie informacje potrzebne do stworzenia B.

### Przykład: Bike jako ekspert

In [ ]:
class Bike:
    def __init__(self, wheel_width, frame_length):
        # Bike jest ekspertem - ma wszystkie dane
        self.__wheel = Wheel(wheel_width)  # <- width dostarczone
        self.__frame = Frame(frame_length)  # <- length dostarczone

**Bike jest ekspertem**, bo:
- Dostaje `wheel_width` i `frame_length`
- Wie jak stworzyć `Wheel(width)` i `Frame(length)`

### Przykład: Obliczanie całkowitej ceny zamówienia

In [6]:
class Product:
    def __init__(self, name, price):
        self.name = name
        self.price = price

class OrderItem:
    def __init__(self, product: Product, quantity: int):
        self.product = product
        self.quantity = quantity
    
    def get_subtotal(self):
        # OrderItem jest ekspertem - zna product i quantity
        return self.product.price * self.quantity

class Order:
    def __init__(self):
        self.items = []
    
    def add_item(self, item: OrderItem):
        self.items.append(item)
    
    def get_total(self):
        # Order jest ekspertem - zna wszystkie items
        return sum(item.get_subtotal() for item in self.items)


In [7]:
# client code
product1 = Product("Laptop", 3000)
product2 = Product("Mysz", 50)

order = Order()
order.add_item(OrderItem(product1, 1))
order.add_item(OrderItem(product2, 2))

print(f"Całkowity koszt: {order.get_total()} zł")

Całkowity koszt: 3100 zł


**Podział odpowiedzialności**:
- `OrderItem.get_subtotal()` - ekspert w obliczaniu subtotal (zna product + quantity)
- `Order.get_total()` - ekspert w obliczaniu total (zna wszystkie items)

**Złe rozwiązanie** (łamie Information Expert):
```python
class Order:
    def get_total(self):
        # Order nie powinien znać szczegółów Product!
        total = 0
        for item in self.items:
            total += item.product.price * item.quantity  # <- Złe!
        return total
```

Problem: `Order` musi znać strukturę `Product` i `OrderItem`, bo sam tych informacji nie posiada.

---

### Problem: Information Expert vs God Object

**Scenariusz**: Dodajmy system zniżek do zamówienia.
- Jeśli kupujesz 2+ produkty typu A, dostajesz 10% zniżki
- Jeśli łączna wartość > 5000 zł, dostajesz 5% zniżki
- Zniżki się kumulują

**Gdzie umieścić logikę zniżek?**

In [8]:
# Źle: God Object
class Order:
    def __init__(self):
        self.items = []
    
    def add_item(self, item):
        self.items.append(item)
    
    def get_total(self):
        total = sum(item.get_subtotal() for item in self.items)
        
        # Zniżka za ilość
        product_counts = {}
        for item in self.items:
            product_counts[item.product.name] = product_counts.get(item.product.name, 0) + item.quantity
        
        for product_name, count in product_counts.items():
            if count >= 2:
                total *= 0.9  # 10% zniżki
        
        # Zniżka za wartość
        if total > 5000:
            total *= 0.95  # 5% zniżki
        
        return total

# Problem: Order wie o WSZYSTKIM (God Object)
# - Zna strukturę Item
# - Zna zasady zniżek
# - Liczy ilości produktów
# - Stosuje wszystkie zniżki
# Każda zmiana wymaga edycji Order

**Lekcja**: Information Expert to balans.
- Order ma dane (items), więc może liczyć total
- Ale zasady zniżek to **osobna odpowiedzialność** → deleguj do Discount

**Nie wkładaj wszystkiego w jedną klasę tylko dlatego, że ma dane.**

In [9]:
# Dobrze: Delegacja do ekspertów
from abc import ABC, abstractmethod

class Discount(ABC):
    @abstractmethod
    def apply(self, total: float, order) -> float:
        pass

class QuantityDiscount(Discount):
    def apply(self, total: float, order) -> float:
        product_counts = {}
        for item in order.items:
            product_counts[item.product.name] = product_counts.get(item.product.name, 0) + item.quantity
        
        for count in product_counts.values():
            if count >= 2:
                return total * 0.9
        return total

class ValueDiscount(Discount):
    def apply(self, total: float, order) -> float:
        if total > 5000:
            return total * 0.95
        return total

class Order:
    def __init__(self, discounts: list[Discount]):
        self.items = []
        self.discounts = discounts
    
    def add_item(self, item):
        self.items.append(item)
    
    def get_total(self):
        total = sum(item.get_subtotal() for item in self.items)
        
        # Delegacja do ekspertów
        for discount in self.discounts:
            total = discount.apply(total, self)
        
        return total


In [10]:
# cilent code
order = Order([QuantityDiscount(), ValueDiscount()])
# Nowa zniżka = nowa klasa, nie edycja Order

## Związek Creator + Information Expert

**Information Expert** to warunek dla **Creator**:

```
Creator: "Bike powinien tworzyć Wheel"
       ↓
Information Expert: "Bike musi mieć wszystkie dane do stworzenia Wheel"
```

**Przykład**:
```python
class Bike:
    def __init__(self, wheel_width):  # <- Information Expert (ma dane)
        self.__wheel = Wheel(wheel_width)  # <- Creator (tworzy)
```

Jeśli **nie masz danych** - nie twórz obiektu:
```python
# Źle - Bike nie ma danych o Engine
class Bike:
    def __init__(self):
        self.engine = Engine(power=100)  # <- Skąd 100? To nie Bike!
```

## Podsumowanie

### Creator
- Obiekt, który **składa się z** / **używa** / **zapisuje** inny obiekt, powinien go **tworzyć**
- Upraszcza API dla klienta
- Centralizuje logikę tworzenia

### Information Expert
- Obiekt może tworzyć/przetwarzać tylko to, o czym **ma pełną wiedzę**
- Odpowiedzialność przypisuj tam, gdzie są dane
- Unikaj "długich łańcuchów" (`order.item.product.price`)

### Pytanie diagnostyczne
"Kto powinien tworzyć obiekt X?"
1. Kto **składa się** z X? (Creator)
2. Kto **ma wszystkie dane** do stworzenia X? (Information Expert)

**Związek z SOLID**:
- Creator = kontrola życia obiektów (Composition)
- Information Expert = SRP (odpowiedzialność tam, gdzie dane)